In [ ]:
import pandas as pd
import concurrent.futures
import re
import json
from langchain_community.chat_models import ChatOllama 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
import chardet
import os
from pathlib import Path
from typing import List
from langchain.docstore.document import Document as LangDocument

MASTER_JS = r"C:\users\tuf\documents\cloudysys_nickfury\dcmsln\dcmsln_201812\Master\PageJS\Query"
BUSINESSRULE_DIR = r"C:\Users\TUF\Documents\cloudysys_nickfury\dcmsln\BusinessRule\Commons"
VECTOR_STORE_PATH = "./common/vectorstore"
EMBEDDING_MODEL = "nomic-embed-text"


In [ ]:
# RAG function
def remove_simplified_chinese(text: str) -> str:
    return re.sub(r"[^\x00-\x7F\u4e00-\u9fff\n\r\t\w\s.,:;!?(){}[\]\"'@#$%^&*\-+=\\/]", "", text)

def read_file_with_detected_encoding(file_path: str) -> str:
    print(file_path)

    with open(file_path, 'rb') as f:
        raw = f.read()
    return raw.decode('GB2312', errors='ignore')

def extract_vb_functions(file_path: str) -> List[LangDocument]:
    raw_code = read_file_with_detected_encoding(file_path)

    # 移除簡體中文註解
    cleaned_code = remove_simplified_chinese(raw_code)

    pattern = r"(Public\s+(Sub|Function)|Private\s+(Sub|Function)|Sub|Function)\s+\w+\s*\(.*?\)[\s\S]+?End\s+(Sub|Function)"
    matches = re.finditer(pattern, cleaned_code, re.MULTILINE | re.IGNORECASE)

    docs = []
    file_name = Path(file_path).name

    for match in matches:
        full_func = match.group(0).strip()

        # 擷取函式名稱
        func_name_match = re.search(r"(Sub|Function)\s+(\w+)", full_func)
        func_name = func_name_match.group(2) if func_name_match else "unknown"

        doc = LangDocument(
            page_content=full_func,
            metadata={
                "source": file_name,
                "function": func_name
            }
        )
        docs.append(doc)

    return docs
def collect_all_vb_chunks(directory: str) -> List[LangDocument]:
    all_docs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".vb"):
                file_path = os.path.join(root, file)
                chunks = extract_vb_functions(file_path)
                all_docs.extend(chunks)
    return all_docs

def build_vector_store(docs: List[LangDocument]):
    print(f"[info] Building vectorstore with {len(docs)} code chunks...")
    embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)
    db = FAISS.from_documents(docs, embeddings)
    os.makedirs(VECTOR_STORE_PATH, exist_ok=True)
    db.save_local(VECTOR_STORE_PATH)
    print(f"[success] Vectorstore saved to: {VECTOR_STORE_PATH}")


In [ ]:
# 建立RAG
# all_chunks = collect_all_vb_chunks(SOURCE_DIR)
# build_vector_store(all_chunks)

In [ ]:
class MessageClassification(BaseModel):
    line: int = Field(...)
    original: str = Field(...)
    issue: str = Field(...)
    replacement: str = Field(...)
    reason: str = Field(...)

# Json 輸出格式解析器
parser = JsonOutputParser()
format_instructions = parser.get_format_instructions()

# 使用本地 LLM 模型
embed_model = OllamaEmbeddings(model="nomic-embed-text") 
fst_llm = ChatOllama(model="llama3:8B") #提出
sec_llm = ChatOllama(model="mistral:7B") #審查
third_llm = ChatOllama(model="phi3:3.8B") #檢查改動是否符合原始邏輯，若符合則發動投票

fst_prompt = PromptTemplate.from_template("""
You are a JavaScript expert whose job is to refactor outdated or deprecated syntax into modern, browser-compatible JavaScript.

Let's carefully analyze the provided JavaScript code snippet. Step by step, identify any lines that use deprecated or non-recommended patterns. For each problematic line, explain why it is an issue, and suggest an improved modern alternative.

=== Original Code ===
{code}
===

Think step by step. Then return your output in **valid JSON format** (double quotes only), suitable for machine parsing. (if there are no issues, return an empty array):
**All values must be wrapped in double quotes as string**
**All values except last one must has common in the end**
                                          
[
  {{
    "line": "<line number where the issue occurs>",
    "original": "<original line of code>",
    "issue": "<description of the issue>",
    "replacement": "<modernized line of code>",
    "reason": "<explanation of why this change is needed>"
  }}
]
""")

sec_prompt = PromptTemplate.from_template("""
You are an expert in JavaScript Linter tools.

Carefully review the following proposed code modifications described in JSON format.  
For each item, think step-by-step about the following:

1. **Syntax Validity** – Is the proposed replacement syntactically correct?
2. **Modern Browser Compatibility** – Is the suggested code aligned with current JavaScript standards and compatible with modern browsers?
3. **Practical Feasibility** – Would this change work reliably in real-world JavaScript environments?

Take your time to reason through each point before making your decision.

=== Proposal ===
{proposal}
===

Please return a JSON array. Each item should include your judgment and reasoning in the following format:
[
  {{
    "line": <line number of the proposal>,
    "vote": "approve" | "reject",
    "comment": "<Step-by-step explanation of your reasoning and conclusion>"
  }}
]
""")

third_prompt = PromptTemplate.from_template("""
You are an expert in semantic consistency analysis.

Your task is to compare the original JavaScript code with its rewritten version and determine whether the rewritten code preserves the same logic and functionality.

Follow these steps in your reasoning:
1. Identify the purpose and behavior of the original code.
2. Analyze the rewritten code line by line to understand its logic.
3. Compare both implementations for semantic equivalence.
4. Look for any missing functionality, altered logic, or behavior changes.

=== Original Code ===
{original}

=== Rewritten Code ===
{rewritten}

Based on your analysis, return a JSON object in the following format:
{
  "consistency": "high" | "medium" | "low",
  "comment": "<Does the rewritten code preserve the original intent? Are there any discrepancies or omissions?>"
}
""")


In [ ]:
# 生成.js
def write_modified_js(original_path: str, original_code: str, approved_changes: List[dict]):
    modified_code = original_code
    for item in approved_changes:
        modified_code = modified_code.replace(item["original"], item["replacement"])
    
    p = Path(original_path)
    output_path = p.with_name(f"{p.stem}_modified{p.suffix}")
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(modified_code)

# 審核json
def extract_json(text: str) -> list:
    try:
        # 找第一個 "[" 與最後一個 "]"
        start = text.find('[')
        end = text.rfind(']') + 1

        if start == -1 or end <= start:
            print("找不到 JSON 陣列")
            return []

        json_str = text[start:end]
        json_str = json_str.encode('utf-8').decode('unicode_escape')

        # 嘗試修剪
        json_str = re.sub(r',\s*}', '}', json_str)
        json_str = re.sub(r',\s*]', ']', json_str)

        print("json陣列:", json_str)

        return json.loads(json_str)

    except json.JSONDecodeError as e:
        print(f"\nJSON 解析失敗：{e}")
        print("原始輸出：", text)
        return []

# 呼叫LLM
def call_llm(llm, prompt):
    response = llm.invoke(prompt)
    return response.content if hasattr(response, "content") else response

# ====== Main LLM Logic ======
def analyze_message_with_multi_llm(code: str):
    # Agent 1：提案改寫
    fst_input = fst_prompt.format(code=code)
    fst_response = call_llm(fst_llm, fst_input)
    proposals = extract_json(fst_response)

    # 印出反饋
    print("fst_output:", proposals)

    if not proposals:
        return {"status": "no-change", "original": code}

    approved_results = []

    for p in proposals:
        # Agent 2：語法審查
        sec_input = sec_prompt.format(proposal=json.dumps([p], ensure_ascii=False))
        sec_result = extract_json(call_llm(sec_llm, sec_input))
        # 印出反饋
        print("sec_result:", sec_result)
        

        # Agent 3：語意一致性
        third_input = third_prompt.format(
            original=p["original"],
            rewritten=p["replacement"]
        )
        third_result = extract_json(call_llm(third_llm, third_input))
        # 印出反饋
        print("third_result:", third_result)

        # 投票機制整合
        vote = (
            (sec_result and sec_result[0]["vote"] == "approve")
            and (third_result and third_result["consistency"] in ["high", "medium"])
        )

        if vote:
            approved_results.append(p)

    return {"status": "done", "approved": approved_results}

# ====== Folder Runner ======

def run_folder_review(folder_path: str):
    js_files = list(Path(folder_path).rglob("*.js"))
    print(f"共發現 {len(js_files)} 個 JS 檔案")

    for file_path in js_files:
        print(f"\n 分析：{file_path}")
        with open(file_path, "r", encoding="utf-8") as f:
            code = f.read()

        result = analyze_message_with_multi_llm(code)

        if result["status"] == "done":
            write_modified_js(file_path, result["original"], result["approved"])
        else:
            print(f"無需修改：{file_path}")

# 單一個js FOR測試
def run_first_js_review(folder_path: str):
    js_files = list(Path(folder_path).rglob("*.js"))
    if not js_files:
        print("找不到任何 JS 檔案")
        return

    first_file = js_files[0]
    print(f"正在分析第一個 JS 檔案：{first_file}")

    with open(first_file, "r", encoding="utf-8") as f:
        code = f.read()

    result = analyze_message_with_multi_llm(code)

    if result["status"] == "done":
        write_modified_js(first_file, result["original"], result["approved"])
    else:
        print(f"無需修改：{first_file}")


In [ ]:
run_first_js_review(MASTER_JS)